# RSD PACKING OF ELONGATED PHYSICAL LINKS
The code adopts Numba to speeding up computation translating to native C++
There are 3 main functions to deposit packings: 

1) RSD_3D_rods_slanted_deposit_fixedlength: which deposits physical links (cylinders) between the opposite faces of a unit cubic box. Links have fixed projection length, r, and are therefore obtained by selecting at random an availabile position on the z=0 unit square and then picking a random angle, theta, to determine the position of the top endpoint on the z=1 unit square;
2) RSD_3D_rods_slanted_deposit_PBC: performing the same process in 1 only usin now periodic boundary conditions on the external faces of the unit box; 
3) RSD_3D_rods_rods_PBC_linklink_jit: depositing elongated physical links in the unit box without the constraint that the endpoints of the links have to lie on the bottom and top plane of the unit box. The code uses periodic boundary conditions in 3D in order to avoid boundary effects. 

In [5]:
import numpy as np
from scipy.linalg import norm
import random as rm
import networkx as nx
import math
from numba import jit, njit #jit with nopython=True
import time


''' ____________ Segment-segment reduction to 2D distances ____________ '''

# S1: endpoints of segment as a single numpy array with shape (6,), i.e., [x1,y1,z1,x2,y2,z2]
# S2: endpoints of segment as a single numpy array with shape (6,), i.e., [x1,y1,z1,x2,y2,z2]
# returns the minimum distance between two given segments
# Based on http://geomalgorithms.com/a07-_distance.html#dist3D_Segment_to_Segment

def dist_slanted_T(S1, S2): 
    S1P0, S1P1 = S1[:2], S1[3:] 
    S2P0, S2P1 = S2[:2], S2[3:]
    X_12 = S2P1[0] - S1P1[0] - S2P0[0] + S1P0[0]
    Y_12 = S2P1[1] - S1P1[1] - S2P0[1] + S1P0[1]
    num = (S1P0[0] - S2P0[0]) * X_12 + (S1P0[1] - S2P0[1]) * Y_12
    den = X_12**2 + Y_12**2
    return num / den
dist_slanted_T_jit = njit()(dist_slanted_T)

def dist_slice_sq(S1, S2, t): 
    S1P0, S1P1 = S1[:2], S1[3:] 
    S2P0, S2P1 = S2[:2], S2[3:]
    x_dist = (S2P0[0] - S1P0[0]) * (1 - t) + (S2P1[0] - S1P1[0]) * t
    y_dist = (S2P0[1] - S1P0[1]) * (1 - t) + (S2P1[1] - S1P1[1]) * t
    return x_dist ** 2 + y_dist ** 2
dist_slice_sq_jit = njit()(dist_slice_sq)

def dist3D_Segment_to_Segment_slanted(S1, S2): 
    if dist_slice_sq_jit(S1, S2, 0) == dist_slice_sq_jit(S1, S2, 1):
        dist3D_S1S2 = dist_slice_sq_jit(S1, S2, 0) # if parallel
    else: 
        T = dist_slanted_T_jit(S1, S2)
        if T >= 1.0: 
            dist3D_S1S2 = dist_slice_sq_jit(S1, S2, 1.0)
        elif T <= 0.0: 
            dist3D_S1S2 = dist_slice_sq_jit(S1, S2, 0.0)
        else:
            dist3D_S1S2 = dist_slice_sq_jit(S1, S2, T) 
    return dist3D_S1S2

# The jitted version takes about 580 ns / pair
dist3D_Segment_to_Segment_slanted_jit = njit()(dist3D_Segment_to_Segment_slanted)




''' ***** 3D RSD of rods - Fixed length rods with closed BCs ***** ''' 

def RSD_3D_rods_slanted_deposit_fixedlength(S, r, d_ex, d_ex_sq, n_max):    
    
    n = 0
    # If the conflict list is needed uncomment the line below
    #conf_list = [] # list of fibers' interactions for each attempted deposition
    
    while n <= n_max: 
        
        theta = 2 * np.pi * rm.uniform(0.0, 1.0)
        
        x1_temp, y1_temp, z1_temp = rm.uniform(0, 1 - d_ex), rm.uniform(0, 1 - d_ex), 0.0
        x2_temp, y2_temp, z2_temp = x1_temp + r * np.cos(theta), y1_temp + r * np.sin(theta), 1.0
        S_temp = np.array( (x1_temp, y1_temp, z1_temp, x2_temp, y2_temp, z2_temp) )
        
        cond = (d_ex <= x2_temp <= 1 - d_ex and d_ex <= y2_temp <= 1 - d_ex)      
        
        ''' ======= Distances between rods in the box ======= '''
        
        distances = []
        for j in range(len(S)):  
            distances.append(dist3D_Segment_to_Segment_slanted_jit(S_temp, S[j]))
                    
        n += 1    
        # Saving here how many conflicts this deposition would have had
        #conf_list.append(len([d for d in distances if d < 4 * d_ex_sq]))
        
        if (min(distances) >= 4 * d_ex_sq and cond == True): 
            S.append(S_temp)
            break 
        
    # If the conflict list is needeed uncomment the line below
    #return S, conf_list, n
    
    return S, n

RSD_3D_rods_slanted_deposit_fixedlength_jit = njit()(RSD_3D_rods_slanted_deposit_fixedlength)

In [4]:
N = 10**6 # number of sequentially displaced segments
d_ex = 0.0005 # disk radius at the density rho_max_est
d_ex_sq = d_ex ** 2
n_max = 10**9 # maximum number of attempted depositions in a row
r = 0.5 #tunes from 2D to 3D model
path = '/Users/ivan_bonamassa/Desktop/'

waiting = [0]
x1, y1 = rm.uniform(d_ex, 1 - d_ex), rm.uniform(d_ex, 1 - d_ex)
S = [ np.array((x1, y1, 0.0, x1, y1, 1.0)) ] # first is planted parallel 

i = 1
while i < N: 

    i += 1
    S, n = RSD_3D_rods_slanted_deposit_fixedlength_jit(S, r, d_ex, d_ex_sq, n_max)
    print(i, S[-1], n)

    waiting.append(n)
    if n >= n_max:
        break

    np.savetxt(path+'_lam'+str(round(d_ex, 6))+'lim.txt', np.c_[S, waiting])


2 [0.16986696 0.32689377 0.         0.32884361 0.80094695 1.        ] 4
3 [0.22111801 0.36605882 0.         0.65392524 0.61641438 1.        ] 1
4 [0.15379025 0.15005495 0.         0.64038425 0.26506051 1.        ] 6
5 [0.29093355 0.13276895 0.         0.33555736 0.63077369 1.        ] 2
6 [0.35665687 0.32493901 0.         0.85484448 0.28240535 1.        ] 1
7 [0.1347404  0.51300324 0.         0.4247882  0.10572904 1.        ] 5
8 [0.78657549 0.71276351 0.         0.37323545 0.99409962 1.        ] 1
9 [0.30832449 0.60221986 0.         0.80594305 0.6509617  1.        ] 2
10 [0.88411974 0.12352788 0.         0.81813987 0.61915542 1.        ] 4
11 [0.03910741 0.66491095 0.         0.46387633 0.40114712 1.        ] 1
12 [0.29229837 0.68077945 0.         0.35825471 0.18514878 1.        ] 1
13 [0.54260911 0.0730117  0.         0.9190708  0.4020658  1.        ] 1
14 [0.83742322 0.78178912 0.         0.45254887 0.4626156  1.        ] 4
15 [0.57594696 0.89183979 0.         0.15681452 0.61920818 

In [3]:

''' +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ '''
'''            BRODS - 3D Random Sequential Deposition (( PBCs ))           '''
''' +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ '''
# The PBC distance function works here for r<=1/2.
# For larger lenghts, one has to evaluate also the virtual replicas in each replicated box

def RSD_3D_rods_slanted_deposit_PBC(S, r, d_ex, d_ex_sq, n_max):    
    
    n = 0
    # If the conflict list is needed uncomment the line below
    #conf_list = [] # list of fibers' interactions for each attempted deposition
    
    while n <= n_max: 
        
        x1_temp, y1_temp, z1_temp = rm.uniform(0, 1), rm.uniform(0, 1), 0.0
        
        theta = 2 * np.pi * np.random.uniform(0, 1) 
        #theta = 2 * np.pi * (2 * np.random.beta(1.2, 1.2) + np.random.beta(.15, .15)) / 3 #initial, man-like bump
        #theta = np.pi * (np.random.beta(1, 1) + np.random.beta(0.25, 0.25))  #initial, pagoda-like bump
        
        R = r # Rods have the same length
        #R = rm.uniform(0.0, r) # Rods have lengths uniformly sampled in [0, r]
        #R = rm.gammavariate(2, r) #Rods' lenghts are gamma sampled from [0, bound]
        
        x2_temp, y2_temp, z2_temp = x1_temp + R * np.cos(theta), y1_temp + R * np.sin(theta), 1.0
        S_temp = np.array( (x1_temp, y1_temp, z1_temp, x2_temp, y2_temp, z2_temp) )
        
        ''' ======= Distances between rods in the box ======= '''
        
        distances = []
        for j in range(len(S)):  
            distances.append(dist3D_Segment_to_Segment_slanted_jit(S_temp, S[j]))
        
        
        ''' ======= Distances between rods due to PBCs ======= '''
        
        S1 = [a + np.array((1.0, 0.0, 0.0, 1.0, 0.0, 0.0)) for a in S]
        S2 = [a + np.array((-1.0, 0.0, 0.0, -1.0, 0.0, 0.0)) for a in S]
        S3 = [a + np.array((0.0, 1.0, 0.0, 0.0, 1.0, 0.0)) for a in S]
        S4 = [a + np.array((0.0, -1.0, 0.0, 0.0, -1.0, 0.0)) for a in S]
        S13 = [a + np.array((1.0, 1.0, 0.0, 1.0, 1.0, 0.0)) for a in S]
        S14 = [a + np.array((1.0, -1.0, 0.0, 1.0, -1.0, 0.0)) for a in S]
        S23 = [a + np.array((-1.0, 1.0, 0.0, -1.0, 1.0, 0.0)) for a in S]
        S24 = [a + np.array((-1.0, -1.0, 0.0, -1.0, -1.0, 0.0)) for a in S]
        
        for j in range(len(S)):
            
            distances.append(dist3D_Segment_to_Segment_slanted_jit(S_temp, S1[j]))
            distances.append(dist3D_Segment_to_Segment_slanted_jit(S_temp, S2[j]))
            distances.append(dist3D_Segment_to_Segment_slanted_jit(S_temp, S3[j]))
            distances.append(dist3D_Segment_to_Segment_slanted_jit(S_temp, S4[j]))
            distances.append(dist3D_Segment_to_Segment_slanted_jit(S_temp, S13[j]))
            distances.append(dist3D_Segment_to_Segment_slanted_jit(S_temp, S14[j]))
            distances.append(dist3D_Segment_to_Segment_slanted_jit(S_temp, S23[j]))
            distances.append(dist3D_Segment_to_Segment_slanted_jit(S_temp, S24[j]))     
            
        n += 1    
        # Saving here how many conflicts this deposition would have had
        #conf_list.append(len([d for d in distances if d < 4 * d_ex_sq]))
        
        if min(distances) >= 4 * d_ex_sq: # notice that dist3D_slanted is a dist_squared
            S.append(S_temp)
            break 
        
    # If the conflict list is needeed uncomment the line below
    #return S, conf_list, n
    
    return S, n

RSD_3D_rods_slanted_deposit_PBC_jit = njit()(RSD_3D_rods_slanted_deposit_PBC)

In [ ]:
# *************** PBC Deposition snippet and saving data  *************** 

N = 10**6
d_ex = 1e-3 # disk radius at the density rho_max_est
d_ex_sq = d_ex ** 2
r = 0.25 # additional projection to the (x1, y1) (rods' fixed lenght)

n_max = 10**9
path = '/Users/ivan_bonamassa/Desktop/'

waiting = [0]

theta = 2 * np.pi * np.random.beta(2, 5)
x1, y1, z1 = rm.uniform(0, 1), rm.uniform(0, 1), 0.0
x2, y2, z2 = x1 + r * np.cos(theta), y1 + r * np.sin(theta), 1.0

S = [ np.array((x1, y1, z1, x2, y2, z2)) ]

i = 1
while i < N: 

    i += 1
    S, n = RSD_3D_rods_slanted_deposit_PBC_jit(S, r, d_ex, d_ex_sq, n_max)
    print(i, S[-1], n)

    waiting.append(n)
    if n >= n_max:
        break

    np.savetxt(path+'r025_beta25_lam'+str(round(d_ex, 10))+'lim.txt', np.c_[S, waiting])
    
'''
# ================= Growing the sample after the initial configuration =============
from physnet_fun import * 

N = 10**5 # number of sequentially displaced segments
d_ex = 1e-4 # disk radius at the density rho_max_est
d_ex_sq = d_ex ** 2
n_max = 10**9 # maximum number of attempted depositions in a row
r = 0.5 #tunes from 2D to 3D model

path = '/Users/ivan_bonamassa/Desktop/RSD_3D_PBCr_betasampling/'
data = np.loadtxt('/Users/ivan_bonamassa/Desktop/RSD_3D_PBCr_betasampling/r05_02_beta025025_lam0.0001lim.txt')

S, waiting = [], []

for dat in data: 
    S.append(dat[:6])
    waiting.append(dat[6])

i = len(S)
while i < N: 

    i += 1
    S, n = RSD_3D_rods_slanted_deposit_PBC_jit(S, r, d_ex, d_ex_sq, n_max)
    print(i, S[-1], n)

    waiting.append(n)
    if n >= n_max:
        break

    np.savetxt(path+'r05_02_beta025025_lam'+str(round(d_ex, 6))+'lim.txt', np.c_[S, waiting])
'''    

## 3D elongated rods in the unit box (no bipartite constraint)

In [7]:
''' ************************************************************************** '''
'''     RSD - 3D rods with fixed lenght + PBC - only link-link interactions    '''
''' ************************************************************************** '''

''' ___________ 3D Segment-segment Euclidean distanece (jitted) ___________ '''

# S1: endpoints of segment as a single numpy array with shape (6,), i.e., [x1,y1,z1,x2,y2,z2]
# S2: endpoints of segment as a single numpy array with shape (6,), i.e., [x1,y1,z1,x2,y2,z2]
# returns the minimum distance between two given segments
# Based on http://geomalgorithms.com/a07-_distance.html#dist3D_Segment_to_Segment

def dist3D_Segment_to_Segment(S1, S2):

    SMALL_NUM=1e-6

    S1P0 = S1[:3] # to np.array for jit acceleration
    S1P1 = S1[3:]

    S2P0 = S2[:3]
    S2P1 = S2[3:]

    u = S1P1 - S1P0
    v = S2P1 - S2P0
    w = S1P0 - S2P0

    a = np.dot(u,u)        # always >= 0
    b = np.dot(u,v)
    c = np.dot(v,v)        # always >= 0
    d = np.dot(u,w)
    e = np.dot(v,w)
    D = a*c - b*b;         # always >= 0
    sD = D
    tD = D

    # compute the line parameters of the two closest points
    if D < SMALL_NUM:      # the lines are almost parallel
        sN = 0.0           #  force using point P0 on segment S1
        sD = 1.0           #  to prevent possible division by 0.0 later
        tN = e
        tD = c
    else:                  # get the closest points on the infinite lines
        sN = (b*e - c*d)
        tN = (a*e - b*d)
        if sN < 0.0:       # sc < 0 => the s=0 edge is visible
            sN = 0.0
            tN = e
            tD = c
        elif sN > sD:      # sc > 1  => the s=1 edge is visible
            sN = sD
            tN = e + b
            tD = c

    if tN < 0.0:            # tc < 0 => the t=0 edge is visible
        tN = 0.0
        # recompute sc for this edge
        if -d < 0.0:
            sN = 0.0
        elif -d > a:
            sN = sD
        else:
            sN = -d
            sD = a
    elif tN > tD:      # tc > 1  => the t=1 edge is visible
        tN = tD
        # recompute sc for this edge
        if -d + b < 0.0:
            sN = 0.0
        elif (-d + b) > a:
            sN = sD
        else:
            sN = (-d +  b)
            sD = a

    # finally do the division to get sc and tc
    if np.abs(sN) < SMALL_NUM:
        sc = 0.0
    else:
        sc= sN/sD
    if np.abs(tN) < SMALL_NUM:
        tc = 0.0
    else:
        tc = tN/tD

    # get the difference of the two closest points
    dP = w + (sc * u) - (tc * v)
    
    return np.linalg.norm(dP)   # return the closest distance

dist3D_Segment_to_Segment_jit = njit()(dist3D_Segment_to_Segment)

    
def RSD_3D_rods_rods_PBC_linklink(rods_dep, r, d_ex, n_max, boundaries):
    
    '''    
    Inputs
    rods_dep          Rods successfully deposited in the box
    d_ex:             Radius of links and nodes - bounded by ~ 0.4
    n_max             Impatience, maximum waiting time before breaking the loop
    
    Output:
    rods_dep          updated list of deposited rods
    n                 Waiting time needed to deposit the new rod
    '''        
    n = 0
    while n <= n_max: 
        
        R = r # rods have the same length
        Theta_temp = np.random.uniform(0, 2 * np.pi) # inclination angle
        x1_temp, y1_temp, z1_temp = rm.uniform(0, 1), rm.uniform(0, 1), rm.uniform(0, 1)
        
        # =========== Uniform sampling on S2 ===========
        u_temp = np.random.uniform(-1, 1)
        x2_temp = x1_temp + R * np.cos(Theta_temp) * np.sqrt(1 - u_temp**2)
        y2_temp = y1_temp + R * np.sin(Theta_temp) * np.sqrt(1 - u_temp**2)
        z2_temp = z1_temp + R * u_temp
        
        # =========== Non-uniform sampling (poles bias) ===========
        #Phi_temp = np.random.uniform(0, np.pi) # azimuthal angle
        #x2_temp = x1_temp + R * np.sin(Theta_temp) * np.cos(Phi_temp)
        #y2_temp = y1_temp + R * np.sin(Theta_temp) * np.sin(Phi_temp)
        #z2_temp = z1_temp + R * np.cos(Theta_temp)
        
        rod_temp = np.array( (x1_temp, y1_temp, z1_temp, x2_temp, y2_temp, z2_temp) )
                
        # Computing the distances *************
        distances = []
        
        for b in range(len(boundaries)): 
            rods_depPBC = [a + np.array(boundaries[b] + boundaries[b]) for a in rods_dep]
            
            for j in range(len(rods_depPBC)): 
                # 3D rod packing in the cloud of 3d rods: (link-link), virtual-deposited
                distances.append(dist3D_Segment_to_Segment_jit(rods_depPBC[j], rod_temp))    
        
        n += 1
        if min(distances) >= 2 * d_ex:
            rods_dep.append(rod_temp)
            break
                   
    return rods_dep, n

RSD_3D_rods_rods_PBC_linklink_jit = njit()(RSD_3D_rods_rods_PBC_linklink)



''' # ================  Deposition snippet  ================

from physnet_fun import * 

N = 10**1 # maximal number of sequentially displaced segments
d_ex = 2.5e-2 # disk radius 
n_max = 10**9
r = 0.5

path = '/Users/ivan_bonamassa/Desktop/Frods_3D_PBC/'

dist_3D_01 = -1.0
n = 0
while dist_3D_01 < 2 * d_ex: 

    n += 1
    x1, y1, z1 = rm.uniform(d_ex, 1 - d_ex), rm.uniform(d_ex, 1 - d_ex), rm.uniform(d_ex, 1 - d_ex)

    R = r
    Theta_temp0 = np.random.uniform(0, 2 * np.pi) # inclination angle
    Phi_temp0 = np.random.uniform(0, np.pi) # azimuthal angle

    x2 = x1 + R * np.sin(Theta_temp0) * np.cos(Phi_temp0)
    y2 = y1 + R * np.sin(Theta_temp0) * np.sin(Phi_temp0)
    z2 = z1 + R * np.cos(Theta_temp0)
    rod_temp0 = np.array( (x1, y1, z1, x2, y2, z2) )

    dist_3D_01 = euclid3D_jit(rod_temp0[:3], rod_temp0[3:])

    if n == n_max:
        print(' -- Script aborted -- No space even for 2 end spheres - Use a smaller lambda')
        exit()

waiting = [0]
rods_dep = [rod_temp0]
boundaries = [p for p in itertools.product([-1, 0, 1], repeat=3)]

i = 1
while i <= N:  

    i += 1

    rods_dep, n = RSD_3D_rods_rods_PBC_linklink_jit(rods_dep, r, d_ex, n_max, boundaries)
    print(i, rods_dep[-1], n)

    waiting.append(n)
    #np.savetxt(path+'RSD_3Drods_lam_varlen'+str(round(d_ex, 10))+'.txt', np.c_[rods_dep, waiting])

    if n >= n_max:
        break
        
'''

" # ================  Deposition snippet  ================\n\nfrom physnet_fun import * \n\nN = 10**1 # maximal number of sequentially displaced segments\nd_ex = 2.5e-2 # disk radius \nn_max = 10**9\nr = 0.5\n\npath = '/Users/ivan_bonamassa/Desktop/Frods_3D_PBC/'\n\ndist_3D_01 = -1.0\nn = 0\nwhile dist_3D_01 < 2 * d_ex: \n\n    n += 1\n    x1, y1, z1 = rm.uniform(d_ex, 1 - d_ex), rm.uniform(d_ex, 1 - d_ex), rm.uniform(d_ex, 1 - d_ex)\n\n    R = r\n    Theta_temp0 = np.random.uniform(0, 2 * np.pi) # inclination angle\n    Phi_temp0 = np.random.uniform(0, np.pi) # azimuthal angle\n\n    x2 = x1 + R * np.sin(Theta_temp0) * np.cos(Phi_temp0)\n    y2 = y1 + R * np.sin(Theta_temp0) * np.sin(Phi_temp0)\n    z2 = z1 + R * np.cos(Theta_temp0)\n    rod_temp0 = np.array( (x1, y1, z1, x2, y2, z2) )\n\n    dist_3D_01 = euclid3D_jit(rod_temp0[:3], rod_temp0[3:])\n\n    if n == n_max:\n        print(' -- Script aborted -- No space even for 2 end spheres - Use a smaller lambda')\n        exit()\n\nwai

# Bundling analysis of physical link packings
In the manuscript, we consider two physical links as bundled if they are nearest neighbors in both the bottom and top plane according to a proximity network. To determine the proximity network, we adopted three alternatives, each giving similar results: i) Delaunay triangulation, ii) geometric graphs with diameter 1.5 times larger than the mean distance between the 2D coordinates of the physical links' endpoints, iii) alpha complex. In what below, we provide the function measuring bundled pairs, further counting their nematic and their hexatic order. 

In [1]:
# RGG generated at a given slice on the z (temporal) direction
def gen_RRG(N, R, pos):
    G = nx.random_geometric_graph(N, R, pos=pos)
    return G

# Delaunay triangulation generated at a certain temporal slice
import scipy.spatial
def gen_Del(pos): 
    # Here pos is not a dictionary but a list of (x,y) coord's
    delTri = scipy.spatial.Delaunay(pos) 
    
    edges = set() 
    # for each Delaunay triangle 
    for n in range(delTri.nsimplex): 
    # for each edge of the triangle 
    # sort the vertices 
    # (sorting avoids duplicated edges being added to the set) 
    # and add to the edges set 
        edge = sorted([delTri.vertices[n,0], delTri.vertices[n,1]]) 
        edges.add((edge[0], edge[1])) 
        edge = sorted([delTri.vertices[n,0], delTri.vertices[n,2]]) 
        edges.add((edge[0], edge[1])) 
        edge = sorted([delTri.vertices[n,1], delTri.vertices[n,2]]) 
        edges.add((edge[0], edge[1])) 

    G = nx.Graph(list(edges)) 
        
    return G



''' ======================================================================= '''
'''                       CREATING THE ALPHA COMPLEX                        '''
''' ======================================================================= '''

# Alpha-complex of the Delaunay triangulation at the base
import scipy.spatial


def orientation(pts):
    """ Given an ordered list of points, return  True iff they are given in clockwise order """
    orient = sum(pts[(i + 1) % len(pts)][0] * p[1] - pts[(i + 1) % len(pts)][1] * p[0] for i, p in enumerate(pts))
    return (orient > 0)


#### Voronoi Diagram
def voronoi_diagram(samples, ax=None):
    """
    Compute the Voronoi diagram of the given samples and return the regions inside the convex hull with their centers.
    
    Args:
     * ``samples`` (*matrix of size n_samples x 2*): 2D coordinates of the samples.
     * ``ax`` (*matplotlib axis*): If None, nothing is plotted.
     
    Returns:
     * ``vor_cells`` (*list of (center, region edges)*): Pre-parsed valid and clockwise oriented Voronoi regions.
     * ``vor_ridges`` (*list of (center, region edges)*): Pre-parsed list of edges and associated Voronoi centers.
    """
    # Extract Voronoi regions
    vor = scipy.spatial.Voronoi(samples, qhull_options="Q0")
    # Voronoi Edge index -> Center of Voronoi cells incident to the edge 
    n = len(vor.vertices)
    vor_ridges = {min(vertices) * n + max(vertices): 
                  ((centers[0], vor.points[centers[0]]), (centers[1], vor.points[centers[1]]))
                  for vertices, centers in zip(vor.ridge_vertices, vor.ridge_points) if not -1 in vertices} 
    # Remove "infinite" regions
    vor_cells = [(center, # center
                 vor.vertices[vor.regions[vor.point_region[i]], :], # vertices coordinates
                 vor.regions[vor.point_region[i]],                  # vertices index
                 [min(p, q) * n + max(p, q)                         # edges index
                  for (p, q) in zip(vor.regions[vor.point_region[i]], 
                                    (vor.regions[vor.point_region[i]][1:] + [vor.regions[vor.point_region[i]][0]]))]) 
                for i, center in enumerate(vor.points) 
            if (vor.point_region[i] != -1) and (not -1 in vor.regions[vor.point_region[i]])]
    # Orient each region's edges in clockwise order
    vor_cells = [(center, np.array(region), np.array(region_indices), edges) if orientation(region) else 
                 (center, np.array(region[::-1]), np.array(region_indices[::-1]), edges[:-1][::-1] + [edges[-1]])
                   for center, region, region_indices, edges in vor_cells]
    return vor_cells, vor_ridges


def is_in_circle(p, c, r):
    """ Returns True iff p lies in (or on) the circle of center ``c`` and radius ``r`` """
    return euclid2D(c, p) <= r


### Alpha Complexes
def line_circle_intersection(p, q, center, r):    
    """
    Return [p2, q2] the segment intersection of [p, q] with the circle of center ``center`` and radius ``r``;
    or an empty list if the intersection does not exist.
    """
    cp = is_in_circle(p, center, r)
    cq = is_in_circle(q, center, r)
    # If p and q are both in the circle -> clip to [p, q]
    if cp and cq:
        return [(p, q)], True, True
    # Otherwise, compute the intersection
    # Finite slope
    if p[0] != q[0]:
        order = p[0] <= q[0]
        # Express the line equation as y = slope * x + intersect
        slope = (q[1] - p[1]) / (q[0] - p[0])
        intersect = q[1] - slope * q[0]
        # Express the intersection problem as a quadratic equation ax2 + bx + c
        a = slope**2 + 1
        b = 2 * (slope * (intersect - center[1]) - center[0])
        c = center[0]**2 + (intersect - center[1])**2 - r**2
        # No intersection -> clip to circle
        delta = b**2 - 4*a*c
        if delta <= 0:
            return [], False, False
        # Intersection -> clip to [p2, q2] n [p, q]
        else:
            pt1 = p; pt2 = q
            is_in_pq = lambda z: ((z >= int(order) * p[0] + (1 - int(order)) * q[0]) 
                        and (z <= int(order) * q[0] + (1 - int(order)) * p[0]))
            check = False # check will be True iff [p2, q2] n [p, q] is empty
            if not cp:
                x = (- b - np.sqrt(delta)) / (2 * a) if order else (- b + np.sqrt(delta)) / (2 * a)
                pt1 = np.array([x, slope*x + intersect])
                check = not is_in_pq(x)
            if not cq:
                x = (- b + np.sqrt(delta)) / (2 * a) if order else (- b - np.sqrt(delta)) / (2 * a)
                pt2 = np.array([x, slope*x + intersect])
                check = (check or cp) and (not is_in_pq(x))
            #print p, q, cp, cq, check, order
            return ([], False, False) if check else ([(pt1, pt2)], cp, cq)
    # Infinite slope (same process, for the case of a vertical line)
    else:
        # No intersection
        if p[0] >= center[0] + r or p[0] <= center[0] - r:
             return [], False, False
        else:
            order = p[1] <= q[1]
            pt1 = p; pt2 = q
            is_in_pq = lambda z: ((z >= int(order) * p[1] + (1 - int(order)) * q[1]) 
                        and (z <= int(order) * q[1] + (1 - int(order)) * p[1]))
            check = False
            if not cp:
                y = (-np.sqrt(r**2 - (p[0] - center[0])**2) if order 
                     else np.sqrt(r**2 - (p[0] - center[0])**2)) + center[1]
                pt1 = np.array([p[0], y])
                check = not is_in_pq(y)
            if not cq:
                y = (np.sqrt(r**2 - (p[0] - center[0])**2) if order 
                     else -np.sqrt(r**2 - (p[0] - center[0])**2)) + center[1]
                pt2 = np.array([p[0], y])
                check = check and (not is_in_pq(y))
            return ([], False, False) if check else ([(pt1, pt2)], cp, cq)
    

# Generating the alpha complex of a 2D Voronoi triangulation of points
import networkx as nx

def alpha_complex_graph(samples, r):
    """
    Jointly compute the alpha complex and Voronoi diagram of the 2D samples
    
    Args:
     * ``samples`` (*matrix of size n_samples x 2*): 2D coordinates of the samples.
     * ``r`` (*float*): Radius of the balls.
     
    Returns:
     * ``G``: Alpha-complex graph of the sample for the given R value
     * ``points``: List of 2D points identifying the nodes' coordinates
    """
    vor_regions, vor_ridges = voronoi_diagram(samples)
    # Build the restricted Voronoi diagram
    alpha_complex_indices = []
    for center, region, region_indices, edge_indices in vor_regions:
        restr_region = [] 
        for i, p in enumerate(region):
            q = region[(i + 1) % len(region)]
            inter, clip_p, clip_q = line_circle_intersection(p, q, center, r)
            if len(inter):
                restr_region.extend(inter)
                alpha_complex_indices.append(vor_ridges[edge_indices[i]])
    
    # Building the networkx graph out of the alpha-complex
    c = alpha_complex_indices
    edge_list_alp = []
    points = np.zeros((len(samples), 2))
    for i in range(len(c)): 
        edge_list_alp.append((c[i][0][0], c[i][1][0]))
        points[c[i][0][0]] = c[i][0][1]
        points[c[i][1][0]] = c[i][1][1]
    
    G = nx.Graph(list(edge_list_alp))
    
    #return c  # indices of the alpha-complex edges
    return G, points


  
# Returns the cosine square of the angle between two vectors
def theta_rel2(u, v):
    return (np.dot(u,v) / (np.linalg.norm(u) * np.linalg.norm(v)))**2


# Definition of the local nematic order parameter
def nematic(u, v):
    return (3 * (np.dot(u,v) / (np.linalg.norm(u) * np.linalg.norm(v)))**2 - 1) / 2


# Computing the nematic order via the spectrum of the nematic tensor
def calculate_nematic_order(rods):
    num_rods = len(rods)
    q_tensor = np.zeros((3, 3))

    for rod in rods:
        u = rod[3:] - rod[:3]  # Calculate the direction vector of the rod
        u /= np.linalg.norm(u)  # Normalize the direction vector
        q_tensor += 1.5 * np.outer(u, u) - 0.5 * np.eye(3)

    q_tensor /= num_rods
    largest_eigenvalue = np.max(np.linalg.eigvalsh(q_tensor))

    return largest_eigenvalue


# Computing the k-th Fourier coefficient 
def Fourier_kcoefficient(theta, k): 
    
    #N = len(theta)
    
    coefficients_plus = [np.exp(1j * k * 2 * np.pi * a) for a in theta]
    fourier_coefficient_plus = np.sum(coefficients_plus) 
    coefficients_minus = [np.exp(- 1j * k * 2 * np.pi * a) for a in theta]
    
    fourier_coefficient_minus = np.sum(coefficients_minus) 
    modulus_squared_plus = np.abs(fourier_coefficient_plus) ** 2
    modulus_squared_minus = np.abs(fourier_coefficient_minus) ** 2
    
    return modulus_squared_plus + modulus_squared_minus



# Definition of the local coherence for the Psi6 planar ordering metric
def Psi6_phase(p0, p1): 
    delx = p0[0] - p1[0]
    dely = p0[1] - p1[1]
    z = complex(delx, dely)
    angle = np.angle(z)
    return np.exp(6.0j * angle)


# Generate the trajectory of the physical segments 
def par_old(p1, p2, ind_vec, i, w, t):
    # w = 0 for the x axis, w = 1 for the y axis, w = 2 for the z axis
    return p1[ind_vec[i][0]][w] + (p2[ind_vec[i][1]][w] - p1[ind_vec[i][0]][w]) * t


def par(p1, p2, t):
    return p1 + (p2 - p1) * t


# Generate the dictionary with the rods' directions 
def vers_fun(p1, p2, ind_vec): 
    return {i: (p2[ind_vec[i][1]][0] - p1[ind_vec[i][0]][0], p2[ind_vec[i][1]][1] \
                - p1[ind_vec[i][0]][1], p2[ind_vec[i][1]][2] - p1[ind_vec[i][0]][2]) \
            for i in range(len(ind_vec))}


# Cylinders' volume 
def volume_rods(p1, p2, d_ex):
    return euclid3D(p1, p2) * math.pi * d_ex ** 2


# Rods density in the box (rods as spherocylinders)
def density_rods(p1, p2, ind_vec, d_ex, z_max):
    vol_cyl = 0
    N_seg = len(ind_vec)
    for i in range(N_seg): 
        a = (par(p1, p2, ind_vec, i, 0, 0), par(p1, p2, ind_vec, i, 1, 0), 0)
        b = (par(p1, p2, ind_vec, i, 0, 1), par(p1, p2, ind_vec, i, 1, 1), z_max)
        vol_cyl += euclid3D(a,b)
    vol_occ = math.pi * d_ex**2 * (vol_cyl + (4/3)* N_seg * d_ex) # Cylinders + 2 half spheres
    vol_box = 1 * 1 * (z_max + 2 * d_ex) # # Box volume + half capped spheres for the nodes
    return vol_occ / vol_box


In [2]:
''' ************ Global bundling order function (06.10.22) ************ '''

def glob_bund_evo_GR(S, d_ex, R_ord, k_threshold, ord_threshold, Psi_threshold):
    
    bund_NN_fraction = [] # Saving the fraction of bundled segments 
    
    R = 2 * R_ord * d_ex # Generally it will depend on d_ex (here we are comparing with d_ex=0)
    
    # The proximity geometric graph at "time" t=0 (3rd entry)
    pos = {i: (S[i][0], S[i][1]) for i in range(len(S))}  
    G0 = gen_RRG(len(S), R, pos)

    # The proximity geometric graph at "time" t=1 (3rd entry)
    pos_zmax = {i: (S[i][3], S[i][4]) for i in range(len(S))}  
    G1 = gen_RRG(len(S), R, pos_zmax)

    vers = {i: (S[i][3] - S[i][0], S[i][4] - S[i][1], S[i][5] - S[i][2]) for i in range(len(S))}

    # Assigning to G0.nodes the invariant versors as nodes' states
    for i in G0.nodes():
        G0.nodes[i]['states'] = vers[i]
        G1.nodes[i]['states'] = vers[i]

    ''' +++++++++ Here's the global bundling part ++++++++++ '''
    # Definition: NN segments at z=0 are also NNs at z=z_max (close segments stay close)

    bundling_vec = []
    for i in G0.nodes():
        vec_i = G0.nodes[i]['states']

        ''' +++++++++ Searching NNs at z=0 tha are NNs at z=z_max ++++++++++ ''' 
        NN_i0 = [n for n in G0.neighbors(i)] # NNs of i at the ground floor
        NN_i1 = [n for n in G1.neighbors(i)] # NNs of i at the z_max floor (for the Psi6 metric of the segment i)
        NN_i1_states = [G1.nodes[n]['states'] for n in G1.neighbors(i)] # States (versors) of the NNs of i at the z-max floor
        NN_i = [x for x in NN_i0 if G0.nodes[x]['states'] in NN_i1_states] # Actual segment neighbors of the segment i!

        loc_OP_i = 0
        for j in NN_i: 
            vec_j = G0.nodes[j]['states']
            loc_OP_i += nematic(vec_i, vec_j)

        # Calculating the average Psi_6 metric for the segment i
        loc_Psi6_i0 = 0
        for j0 in NN_i0: 
            loc_Psi6_i0 += Psi6_phase(S[i][0:2], S[j0][0:2])

        loc_Psi6_i1 = 0
        for j1 in NN_i1: 
            loc_Psi6_i1 += Psi6_phase(S[i][3:5], S[j1][3:5])

        loc_Psi6_i = 0
        if len(NN_i0) !=0: 
            loc_Psi6_i = np.abs(loc_Psi6_i0) / len(NN_i0)        
        elif len(NN_i1) !=0: 
            loc_Psi6_i += 0.5 * np.abs(loc_Psi6_i1) / len(NN_i1) # 0.5 to averaging out

        ''' +++++++ Selecting locally ordered segments +++++++ '''
        k_bundl_i = len(NN_i)
        if k_bundl_i > k_threshold:

            # This is the selection of highly oriented segments
            # Use "S_glob_vec[n]" instead of "ord_threshold" if orientation > glob_nem
            if loc_OP_i / k_bundl_i > ord_threshold and loc_Psi6_i > Psi_threshold: 
                bundling_vec.append((i, k_bundl_i, NN_i, loc_OP_i / k_bundl_i, loc_Psi6_i))

    bund_NN_fraction.append(len(bundling_vec))
    
    return bund_NN_fraction


# =========== Global bundling including the hexaticity analysis =============

def glob_bund_evo_Del(S, d_ex, k_threshold, ord_threshold):
    
    bund_NN_Del_fraction = [] # Saving the fraction of bundled segments 
        
    # The proximity Delaunay geometric graph at "time" t=0 (3rd entry)
    pos = [(S[i][0]%1.0, S[i][1]%1.0) for i in range(len(S))]
    G0 = gen_Del(pos)

    # The proximity Delaunay geometric graph at "time" t=1 (3rd entry)
    pos_zmax = [(S[i][3]%1.0, S[i][4]%1.0) for i in range(len(S))]
    G1 = gen_Del(pos_zmax)

    vers = {i: (S[i][3] - S[i][0], S[i][4] - S[i][1], S[i][5] - S[i][2]) for i in range(len(S))}

    # Assigning to G0.nodes the invariant versors as nodes' states
    for i in G0.nodes():
        G0.nodes[i]['states'] = vers[i]
        G1.nodes[i]['states'] = vers[i]

    ''' +++++++++ Here's the global bundling part ++++++++++ '''
    # Definition: NN segments at z=0 are also NNs at z=z_max (close segments stay close)

    bundling_vec = []
    
    for i in G0.nodes():
        vec_i = G0.nodes[i]['states']

        ''' +++++++++ Searching NNs at z=0 tha are NNs at z=z_max ++++++++++ ''' 
        NN_i0 = [n for n in G0.neighbors(i)] # NNs of i at the ground floor
        NN_i1_states = [G1.nodes[n]['states'] for n in G1.neighbors(i)] # States (versors) of the NNs of i at the z-max floor
        NN_i = [x for x in NN_i0 if G0.nodes[x]['states'] in NN_i1_states] # Actual segment neighbors of the segment i!
        NN_i1 = [n for n in G1.neighbors(i)] # NNs of i at the z_max floor (for the Psi6 metric of the segment i)
        
        loc_OP_i = 0
        for j in NN_i: 
            vec_j = G0.nodes[j]['states']
            loc_OP_i += nematic(vec_i, vec_j)


        k_bundl_i = len(NN_i)
        if k_bundl_i > k_threshold:

            # This is the selection of highly oriented segments
            # Use "S_glob_vec[n]" instead of "ord_threshold" if orientation > glob_nem
            if loc_OP_i / k_bundl_i > ord_threshold:
                bundling_vec.append((i, k_bundl_i, NN_i, loc_OP_i / k_bundl_i))

    bund_NN_Del_fraction.append(len(bundling_vec))
    
    return bund_NN_Del_fraction